<h1 style="font-family:Impact,Arial;font-size:50px">MWATS Statistics </h1>
<p> These data were extracted from the Variable and Slow Transients (VAST) pipeline in csv format and then converted to parquet format via the Load.ipynb script. In this notebook we apply filters and average the data on a daily cadence. We then generate a statistics table to explore the data. Some example plots have been included at the end of the script to plot locations of highly variable objects as well as exploring the overall statistics of the dataset. For more information contact: Martin.Bell@uts.edu.au.


## Data Columns
### _data_ table:

- **raw_peak_flux**: flux density [_JY - Janskys_] (or brightness of the observation).

- **source_name**  : unique identifier of a given source (or object). 

- **jd**          : Julian date [_D - days_].

- **ra** : Right Ascension [_Deg - Degrees_].

- **dec** : Declination [_Deg - Degrees_].

- **im_ra**: Image Right Ascension [_Deg - Degrees_] i.e. the RA of the pointing centre of the image from which the measurement was taken. 

- **im_dec**: Image Declination [_Deg - Degrees_] i.e. the Dec of the pointing centre of the image from which the measurement was taken. 

- **distance**: Distance [_Deg - Degrees_]: distance between pointing centre and flux measurement. 

- **gain**: Gain [no units]. Multiplicative factor that has been applied to each image (and flux measurements in image) to correct flux scale. Should be close to 1. 

### _data _avg_ table: 

- **median_flux** : median flux density [_JY - Janskys_] of daily measurements. 

- **std_flux** : standard deviation of daily flux measurements [_JY - Janskys_].

- **mean_jd** : mean Julian date of daily measurements [_D - days_]. 

- **datetime** : pandas datetime object of mean date of observations. 


### _stats_ table: 

- **mean_raw_peak_flux** : mean flux density of all measurements [_JY - Janskys_].	

- **std_raw_peak_flux**	 : standard deviation of all measurements [_JY - Janskys_].

- **Mod** Modulation Index [%].	

- **length**: Length of time series [_N_].
- **sig**: Significance of gradient [grad / fit_error].	
- **grad** : Linear regression fit to the data.	
- **fit_error** : Error on the linear regression. NOTE: error bars were not used in this fit as per Bell et al. 2018, only the scatter on the measurements. 	
- **y_int** : Y intercept of the linear fit.

- **avg_sig**: Significance of gradient [grad / fit_error] for daily median flux measurements.	
- **avg_grad** : Linear regression fit to the data for daily median flux measurements.	

- **avg_fit_error** : Error on the linear regression for daily median flux measurements. NOTE: error bars were not used in this fit as per Bell et al. 2018, only the scatter on the measurements. 	

- **avg_y_int** : Y intercept of the linear fit for daily median flux measurements.


In [ ]:
import pandas as pd
import numpy as np
#from scipy import stats
from scipy.stats import linregress, entropy

import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, rasterize
hv.extension('bokeh')

import datashader as ds
import datashader.transfer_functions as tf
from datashader.colors import inferno

import matplotlib.pyplot as plt
%matplotlib inline

from astropy.coordinates import ICRS, Galactic, FK4, FK5
from astropy import units as u
from astropy.coordinates import SkyCoord
import datetime

import warnings
warnings.simplefilter('ignore')

import time

import nmslib;

In [ ]:
def distance_from_median_pos(df):
    degrees_to_radians = np.pi/180.0
    phi1 = df.dec*degrees_to_radians
    phi2 = df.median_dec*degrees_to_radians

    theta1 = df.ra*degrees_to_radians
    theta2 = df.median_ra*degrees_to_radians
    
    cosine = (np.cos(phi1)*np.cos(phi2)*np.cos(theta1 - theta2) +
           np.sin(phi1)*np.sin(phi2))
    dist_from_centre = np.arccos(cosine)
    return (dist_from_centre/3.142)*180

## Loading the data
The input data file is the feather file created in Load.ipynb

In [ ]:
%%time
data = pd.read_feather('mwats_raw_data_Mar_SQL.fth')

In [ ]:
%%time
# Add in columns for median RA/DEC and flux.
tmp3 = data.groupby('source_id')[['ra','dec','raw_peak_flux']].median().reset_index() 
tmp3.columns = ['source_id', 'median_ra', 'median_dec', 'median_flux']
data = pd.merge(data,
                 tmp3[['source_id', 'median_ra', 'median_dec', 'median_flux']],
                 on='source_id')

In [ ]:
%time
# Add in columns for std RA/DEC and flux.
tmp4 = data.groupby('source_id')[['ra','dec']].std().reset_index() 
tmp4.columns = ['source_id', 'std_ra', 'std_dec']
data = pd.merge(data,
                 tmp4[['source_id', 'std_ra', 'std_dec']],
                 on='source_id')

In [ ]:
%time
# Add offet and flux_offset data columns. 
# offset = distance (in deg) from median position. 
# flux_offset = measured flux minus the median
data['offset'] = distance_from_median_pos(data)
data['flux_offset'] = np.sqrt((data.raw_peak_flux-data.median_flux)**2)

## Filtering: 
Below are the filters that have been applied to the data. These can be modified based on the users requirement. 

- _Distance from pointing centre_: We filter measurements less than 12 degrees from the pointing centre as per Bell et al. 2016, 2018. Before filtering we have 10 million flux measurements and after filtering 6.2 million. It is quite subjective to define the correct distance, but 12 degrees has worked well in the past.  

- _max__flux_: The brightest object to consider in the analysis. Objects > 50 Jy are typically so bright they cause problems (in imaging) and are hard to generate reliable light-curves. 

- _min__length_: The minimum nimber of points to consider in the light-curves. Objects with small numbers of measurements may be spurius and unrealiable.  



In [ ]:
# Filters (some are applied later in the script)
max_dist   = 15.0 # Only use measurements less that 12-15 degrees from pointing centre of the image.
max_flux   = 50.0 # Jy. Only consider sources with a maximum flux of max_flux.
min_flux   = 0.25 # Jy. Only consider sources with a minimum flux of min_flux.
min_length = 35 # Minimum length of time-series to consider. 
max_rms    = 500 # Remove images with RMS greater than this. A few RFI soaked images crept threw. 
min_rms    = 25  # A couple of badly calibrated images need to be removed. 
max_offset = 0.0375 # beam = 3 pixels with width @ 0.75' per pixel (0.75*3)/60 = 0.0375
max_flux_offset = 5000 # flux_offset = |flux - median_flux| = Gets ride of small number of bad fits

## Apply distance from pointing centre filter:

In [ ]:
data = data[data.distance < max_dist]

## Apply rms image filters:

In [ ]:
data = data[data.rms < max_rms]
data = data[data.rms > min_rms]

## Apply position offset filter:

In [ ]:
# Remove data points more than 3 pixels away from median position
data = data[data.offset < max_offset] # beam = 3 pixels with width @ 0.75' per pixel (0.75*3)/60 = 0.0375

## Apply flux offset filter:

In [ ]:
# Remove sources with really large flux offsets. They are just junk.
data = data[data.flux_offset<max_flux_offset]

## Remove sources that have more than one measurement in an image: 

In [ ]:
good_sources = (data
                .groupby(['image_id','source_id'])
                .size().rename('count').to_frame()
                .reset_index().query('count == 1')['source_id']
               ).values
data = data[data.source_id.isin(good_sources)]

## House keeping

In [ ]:
# Set the date as the index
data.set_index('time', inplace=True)

# Remove sources without as source_id
data = data[data.source_id.notnull()]

# Convert source_id to int
data['source_id'] = data.source_id.astype(int)

## Averaging the full dataset
The steps below re-sample the raw data on day timescales.  

### The cell below speeds up the computation of the averaged data.

In [ ]:
%%time 

data_avg = (data.groupby(['source_id', pd.Grouper(freq = '1d')])
                .agg({
                        'raw_peak_flux': ['median', 'std'],
                        'jd': 'mean'
                     })
                .dropna()
           )

data_avg.columns = data_avg.columns.map('_'.join)

data_avg = (data_avg
                .rename(columns={
                    'raw_peak_flux_median': 'median_flux',
                    'raw_peak_flux_std': 'std_flux',
                    'jd_mean': 'mean_jd'
                                })
                .reset_index())


## Generate non-averaged statistics table

In [ ]:
%%time

# Generate a new stats table with the mean flux of the non-averaging sources
stats = data.groupby('source_id')[['ra','dec','raw_peak_flux']].mean().reset_index() 
stats.columns = ['source_id', 'ra', 'dec', 'mean_raw_peak_flux']

# Maybe to replace repeated uses
#flux_by_source = data.groupby('source_id')['raw_peak_flux']

# Stats: std
tmp = data.groupby('source_id')['raw_peak_flux'].std(ddof=1).reset_index()
tmp.columns = ['source_id', 'std_raw_peak_flux']
stats['std_raw_peak_flux'] = tmp['std_raw_peak_flux']

# Do some basic filtering of df to get rid of junky data points
stats = stats[(stats.mean_raw_peak_flux > 0) & (stats.std_raw_peak_flux > 0)]

# Stats: modulation index
stats['Mod'] = (stats.std_raw_peak_flux / stats.mean_raw_peak_flux) * 100 # In percent

# Stats: number of measurements per source. 
tmp2 = data.groupby('source_id')['raw_peak_flux'].count().reset_index()
tmp2.columns = ['source_id', 'length']
stats['length'] = tmp2['length']


'''
# Stats: Flux variability relative to noise 
def inversesqr(x):
    return 1./(x*2)
tmp3 = data.groupby('source_id')['rms'].apply(inversesqr)
tmp3.columns = ['source_id','weight']
tmp4 = data.groupby('source_id')['raw_peak_flux']
tmp4.columns = ['source_id','flux']


# Look for outliers
# warning quantiles are slow!
def q50(x):
    return x.quantile(.50)
def q16(x):
    return x.quantile(.1586)
def q84(x):
    return x.quantile(.8414)
f = {'raw_peak_flux': [q16, q50, q84]}
tmp = data.groupby('source_id').agg(f)
tmp_q50 = data.groupby('source_id')['raw_peak_flux'].quantile(.50).reset_index()
tmp_q50.columns = ['source_id', 'q50_raw_peak_flux']

tmp_flux = tmp.reset_index()['raw_peak_flux']
tmp_q50 = tmp_flux['q50']
tmp_iqr = tmp_flux['q84'] - tmp_flux['q16']
data['is_flux_high'] = data['raw_peak_flux'].sub(tmp_q50).div(tmp_iqr)  > 3.0
#data['is_flux_high'] = data.groupby('source_id')['scaled_flux'].transform(lambda x: x > 3.0)
stats['frac_high'] = data.groupby('source_id')['is_flux_high'].sum() / stats['length']
'''

# Do some astronomy based filtering
stats = stats[(stats.mean_raw_peak_flux > min_flux)
     & (stats.mean_raw_peak_flux < max_flux)
     & (stats.length > min_length)]

filtered_raw_data = data[data.source_id.isin(stats.source_id)]

 ##  Calculate the gradient (non-averaged)

In [ ]:
%%time
# Get the dates and fluxes
dates  = filtered_raw_data.groupby('source_id')['jd'].apply(list)
fluxes = filtered_raw_data.groupby('source_id')['raw_peak_flux'].apply(list)

# Concat time and flux and do the fit
time_flux = pd.concat((dates, fluxes), axis=1)
grad_fit = time_flux.apply(lambda x: linregress(x['jd'], x['raw_peak_flux']), axis=1)

# Calculate all of the terms
grad = grad_fit.map(lambda x: x.slope)
fit_error = grad_fit.map(lambda x: x.stderr)
sig = grad_fit.map(lambda l: l.slope/l.stderr)
y_int = grad_fit.map(lambda x: x.intercept)

stats = stats.assign(sig=sig.values)
stats = stats.assign(grad=grad.values)
stats = stats.assign(fit_error=fit_error.values)
stats = stats.assign(y_int=y_int.values)

## Generate averaged statistics

In [ ]:
# We want to work with the sources in our filtered stats table (i.e. have decent sized light-curves).
filtered_raw_avg_data = data_avg[data_avg.source_id.isin(stats.source_id)]

In [ ]:
%%time
# Get the dates and fluxes
dates  = filtered_raw_avg_data.groupby('source_id')['mean_jd'].apply(list)
fluxes = filtered_raw_avg_data.groupby('source_id')['median_flux'].apply(list)

In [ ]:
%%time
# Concat time and flux and do the fit
time_flux = pd.concat((dates, fluxes), axis=1)
grad_fit = time_flux.apply(lambda x: linregress(x['mean_jd'], x['median_flux']), axis=1)

In [ ]:
%%time
# Calculate all of the terms and add them to the stats table
avg_grad = grad_fit.map(lambda x: x.slope)
avg_fit_error = grad_fit.map(lambda x: x.stderr)
avg_sig = grad_fit.map(lambda l: l.slope/l.stderr)
avg_y_int = grad_fit.map(lambda x: x.intercept)

stats = stats.assign(avg_sig=avg_sig.values)
stats = stats.assign(avg_grad=avg_grad.values)
stats = stats.assign(avg_fit_error=avg_fit_error.values)
stats = stats.assign(avg_y_int=avg_y_int.values)

In [ ]:
stats.set_index('source_id', inplace=True) # The indices of stats and tmp3 and tmp4 must be aligned. Let's set them to source_id

In [ ]:
%%time
# Calculate the modulation index of the daily median flux measurements. 

# Add in the std of all the daily resampled measurements.
tmp3 = filtered_raw_avg_data.groupby('source_id')['median_flux'].std(ddof=1).reset_index()
tmp3.columns = ['source_id', 'avg_std']
tmp3.set_index('source_id', inplace=True)
stats['avg_std'] = tmp3['avg_std']

# Calculate the mean of the median daily flux measurements. 
tmp4 = filtered_raw_avg_data.groupby('source_id')['median_flux'].mean().reset_index()
tmp4.columns = ['source_id', 'avg_median']
tmp4.set_index('source_id', inplace=True)
stats['avg_median'] = tmp4['avg_median']

# Calculate the modulation index
stats['avg_Mod'] = (stats.avg_std / stats.avg_median) * 100 # In percent

In [ ]:
stats.reset_index(inplace=True)
filtered_raw_avg_data.reset_index(inplace=True)
filtered_raw_data.reset_index(inplace=True)

# Nearest neighbours

Using https://github.com/nmslib/nmslib/blob/master/python_bindings/notebooks/search_vector_dense_optim.ipynb
with Euclidean distance.

In [ ]:
source_positions = filtered_raw_data.groupby('source_id')[['ra', 'dec']].mean()
source_positions.shape

In [ ]:
source2idx = {s: i for i, s in enumerate(source_positions.index.values)}
idx2source = {v:k for (k, v) in source2idx.items()}
X = source_positions.values

In [ ]:
# Set index parameters
# These are the most important ones
M = 15
efC = 100

num_threads = 4
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}
print('Index-time parameters', index_time_params)

# Number of neighbors 
K=100

# Space name should correspond to the space name 
# used for brute-force search
space_name='l2'

# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR) 
index.addDataPointBatch(X)

# Create an index
start = time.time()
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC}
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params)

query_qty = X.shape[0]
start = time.time() 
nbrs = index.knnQueryBatch(X, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty))

In [ ]:
# Unpack the neighbours array into a dataframe

source_id = []
n_id1 = []
n_id2 = []
n_id3 = []
n_id4 = []
n_id5 = []

for n in nbrs:
    tmp_id = idx2source[n[0][0]]
    tmp_id1 = idx2source[n[0][1]]
    tmp_id2 = idx2source[n[0][2]]
    tmp_id3 = idx2source[n[0][3]]
    tmp_id4 = idx2source[n[0][4]]
    tmp_id5 = idx2source[n[0][5]]
    
    source_id.append(tmp_id)
    n_id1.append(tmp_id1)
    n_id2.append(tmp_id2)
    n_id3.append(tmp_id3)
    n_id4.append(tmp_id4)
    n_id5.append(tmp_id5)

In [ ]:
neighbours = pd.DataFrame(
    {'source_id': source_id,
     'n_id1': n_id1,
     'n_id2': n_id2,
     'n_id3': n_id3,
     'n_id4': n_id4,
     'n_id5': n_id5,
     'source_id_ra'   : stats.set_index('source_id').loc[source_id].ra,
     'source_id_dec'  : stats.set_index('source_id').loc[source_id].dec,
      })

In [ ]:
neighbours = neighbours.drop("source_id", axis=1)
neighbours = neighbours.reset_index()

In [ ]:
n_id1_ra = stats.set_index('source_id').loc[n_id1].ra
n_id1_dec = stats.set_index('source_id').loc[n_id1].dec
n_id2_ra = stats.set_index('source_id').loc[n_id2].ra
n_id2_dec = stats.set_index('source_id').loc[n_id2].dec
n_id3_ra = stats.set_index('source_id').loc[n_id3].ra
n_id3_dec = stats.set_index('source_id').loc[n_id3].dec
n_id4_ra = stats.set_index('source_id').loc[n_id4].ra
n_id4_dec = stats.set_index('source_id').loc[n_id4].dec
n_id5_ra = stats.set_index('source_id').loc[n_id5].ra
n_id5_dec = stats.set_index('source_id').loc[n_id5].dec

neighbours = neighbours.assign(n_id1_ra=n_id1_ra.values)
neighbours = neighbours.assign(n_id1_dec=n_id1_dec.values)
neighbours = neighbours.assign(n_id2_ra=n_id2_ra.values)
neighbours = neighbours.assign(n_id2_dec=n_id2_dec.values)
neighbours = neighbours.assign(n_id3_ra=n_id3_ra.values)
neighbours = neighbours.assign(n_id3_dec=n_id3_dec.values)
neighbours = neighbours.assign(n_id4_ra=n_id4_ra.values)
neighbours = neighbours.assign(n_id4_dec=n_id4_dec.values)
neighbours = neighbours.assign(n_id5_ra=n_id5_ra.values)
neighbours = neighbours.assign(n_id5_dec=n_id5_dec.values)

In [ ]:
def vectorized_distance_on_unit_sphere(s1_ra, s2_ra, s1_dec, s2_dec):
    degrees_to_radians = np.pi/180.0
    phi1 = s1_dec*degrees_to_radians
    phi2 = s2_dec*degrees_to_radians

    theta1 = s1_ra*degrees_to_radians
    theta2 = s2_ra*degrees_to_radians
    
    cosine = (np.cos(phi1)*np.cos(phi2)*np.cos(theta1 - theta2) +
           np.sin(phi1)*np.sin(phi2))
    dist_from_centre = np.arccos(cosine)
    return (dist_from_centre/3.142)*180

In [ ]:
neighbours['dist_1'] = vectorized_distance_on_unit_sphere(neighbours.source_id_ra, neighbours.n_id1_ra, neighbours.source_id_dec, neighbours.n_id1_dec)
neighbours['dist_2'] = vectorized_distance_on_unit_sphere(neighbours.source_id_ra, neighbours.n_id2_ra, neighbours.source_id_dec, neighbours.n_id2_dec)
neighbours['dist_3'] = vectorized_distance_on_unit_sphere(neighbours.source_id_ra, neighbours.n_id3_ra, neighbours.source_id_dec, neighbours.n_id3_dec)
neighbours['dist_4'] = vectorized_distance_on_unit_sphere(neighbours.source_id_ra, neighbours.n_id4_ra, neighbours.source_id_dec, neighbours.n_id4_dec)
neighbours['dist_5'] = vectorized_distance_on_unit_sphere(neighbours.source_id_ra, neighbours.n_id5_ra, neighbours.source_id_dec, neighbours.n_id5_dec)

In [ ]:
#ax=np.log10(neighbours.dist_0).hist(bins=100, figsize=(10, 5), grid=False, ec='k')
ax=(neighbours.dist_1).hist(bins=100, figsize=(10, 5), grid=False, ec='k', label='Sources')
plt.plot([0.0625,0.0625],[0,6000], 'r-.', label='5 pixels distance')
plt.xlabel('Distance from nearest neighbour (deg)')
plt.legend()

## Filter out sources that are closer than 5 pixels

In [ ]:
# Filter out the nearby sources
close_source = neighbours[neighbours.dist_1 > 0.0625] # > 5 pixels ADD AS PARAMETER ABOVE
stats = stats[stats.source_id.isin(close_source.source_id)] # Do stats table only, leave in place in data tables. 

In [ ]:
ax=(neighbours.dist_1).hist(bins=80, figsize=(10, 5), grid=False, histtype='step', label='Sources')
ax=(neighbours.dist_2).hist(bins=70, figsize=(10, 5), grid=False, histtype='step', label='Sources')
ax=(neighbours.dist_3).hist(bins=100, figsize=(10, 5), grid=False, histtype='step', label='Sources')
ax=(neighbours.dist_4).hist(bins=90, figsize=(10, 5), grid=False, histtype='step', label='Sources')
ax=(neighbours.dist_5).hist(bins=100, figsize=(10, 5), grid=False, histtype='step', label='Sources')
plt.plot([0.0625,0.0625],[0,6000], 'r-.', label='5 pixels distance')
plt.xlabel('Distance from nearest neighbour (deg)')
#plt.xscale('log')
plt.legend()

In [ ]:
for src in [3045]: #stats.source_id:
    nn1 = neighbours[neighbours.source_id==src].iloc[0]["n_id1"]
    print(nn1)
    lc_src = data[data.source_id==src].raw_peak_flux 
    print(lc_src)
    lc_nb1 = data[data.source_id==nn1].raw_peak_flux
    #if lc_src.length != lc_nb1.length:
    #    print(str(src)+": LC not the same length as neighbour!")

### Save the data

In [ ]:
stats.reset_index(inplace=True)
filtered_raw_avg_data.reset_index(inplace=True)
filtered_raw_data.reset_index(inplace=True)
neighbours.reset_index()

In [ ]:
stats.to_feather('stats_table.fth')
filtered_raw_avg_data.to_feather('avg_data.fth')
filtered_raw_data.to_feather('data.fth')
neighbours.to_feather('neighbours.fth')